## Постановка задачи
Загрузим данные, приведем их к числовым, заполним пропуски, нормализуем данные и оптимизируем память.

Разделим выборку на обучающую/проверочную в соотношении 80/20.

Построим модель опорных векторов (SVM) для наиболее оптимального разделения параметров на классы, используем несколько реализаций: линейную (LinearSVC) и через градиентный бустинг (SGDClassifier).

Проведем предсказание и проверим качество через каппа-метрику.

Данные:
* https://video.ittensive.com/machine-learning/prudential/train.csv.gz

Соревнование: https://www.kaggle.com/c/prudential-life-insurance-assessment/

### Подключение библиотек

In [11]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score, confusion_matrix
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier
from sklearn import preprocessing

### Загрузка данных

In [12]:
data = pd.read_csv("https://video.ittensive.com/machine-learning/prudential/train.csv.gz")
print (data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59381 entries, 0 to 59380
Columns: 128 entries, Id to Response
dtypes: float64(18), int64(109), object(1)
memory usage: 58.0+ MB
None


### Предобработка данных

In [13]:
data["Product_Info_2_1"] = data["Product_Info_2"].str.slice(0, 1)
data["Product_Info_2_2"] = pd.to_numeric(data["Product_Info_2"].str.slice(1, 2))
data.drop(labels=["Product_Info_2"], axis=1, inplace=True)
for l in data["Product_Info_2_1"].unique():
    data["Product_Info_2_1" + l] = data["Product_Info_2_1"].isin([l]).astype("int8")
data.drop(labels=["Product_Info_2_1"], axis=1, inplace=True)
data.fillna(value=-1, inplace=True)

### Набор столбцов для расчета

In [14]:
columns_groups = ["Insurance_History", "InsurеdInfo", "Medical_Keyword",
                  "Family_Hist", "Medical_History", "Product_Info"]
columns = ["Wt", "Ht", "Ins_Age", "BMI"]
for cg in columns_groups:
    columns.extend(data.columns[data.columns.str.startswith(cg)])
print (columns)

['Wt', 'Ht', 'Ins_Age', 'BMI', 'Insurance_History_1', 'Insurance_History_2', 'Insurance_History_3', 'Insurance_History_4', 'Insurance_History_5', 'Insurance_History_7', 'Insurance_History_8', 'Insurance_History_9', 'Medical_Keyword_1', 'Medical_Keyword_2', 'Medical_Keyword_3', 'Medical_Keyword_4', 'Medical_Keyword_5', 'Medical_Keyword_6', 'Medical_Keyword_7', 'Medical_Keyword_8', 'Medical_Keyword_9', 'Medical_Keyword_10', 'Medical_Keyword_11', 'Medical_Keyword_12', 'Medical_Keyword_13', 'Medical_Keyword_14', 'Medical_Keyword_15', 'Medical_Keyword_16', 'Medical_Keyword_17', 'Medical_Keyword_18', 'Medical_Keyword_19', 'Medical_Keyword_20', 'Medical_Keyword_21', 'Medical_Keyword_22', 'Medical_Keyword_23', 'Medical_Keyword_24', 'Medical_Keyword_25', 'Medical_Keyword_26', 'Medical_Keyword_27', 'Medical_Keyword_28', 'Medical_Keyword_29', 'Medical_Keyword_30', 'Medical_Keyword_31', 'Medical_Keyword_32', 'Medical_Keyword_33', 'Medical_Keyword_34', 'Medical_Keyword_35', 'Medical_Keyword_36', 'M

### Нормализация данных

In [15]:
scaler = preprocessing.StandardScaler()
data_transformed = pd.DataFrame(scaler.fit_transform(pd.DataFrame(data,
                                            columns=columns)))
columns_transformed = data_transformed.columns
data_transformed["Response"] = data["Response"]

### Оптимизация памяти

In [16]:
def reduce_mem_usage (df):
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if str(col_type)[:5] == "float":
            c_min = df[col].min()
            c_max = df[col].max()
            if c_min > np.finfo("f2").min and c_max < np.finfo("f2").max:
                df[col] = df[col].astype(np.float16)
            elif c_min > np.finfo("f4").min and c_max < np.finfo("f4").max:
                df[col] = df[col].astype(np.float32)
            else:
                df[col] = df[col].astype(np.float64)
        elif str(col_type)[:3] == "int":
            c_min = df[col].min()
            c_max = df[col].max()
            if c_min > np.iinfo("i1").min and c_max < np.iinfo("i1").max:
                df[col] = df[col].astype(np.int8)
            elif c_min > np.iinfo("i2").min and c_max < np.iinfo("i2").max:
                df[col] = df[col].astype(np.int16)
            elif c_min > np.iinfo("i4").min and c_max < np.iinfo("i4").max:
                df[col] = df[col].astype(np.int32)
            elif c_min > np.iinfo("i8").min and c_max < np.iinfo("i8").max:
                df[col] = df[col].astype(np.int64)
        else:
            df[col] = df[col].astype("category")
    end_mem = df.memory_usage().sum() / 1024**2
    print('Потребление памяти меньше на', round(start_mem - end_mem, 2), 'Мб (минус', round(100 * (start_mem - end_mem) / start_mem, 1), '%)')
    return df

In [17]:
data_transformed = reduce_mem_usage(data_transformed)
print (data_transformed.info())

Потребление памяти меньше на 45.59 Мб (минус 75.1 %)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59381 entries, 0 to 59380
Columns: 134 entries, 0 to Response
dtypes: float16(133), int8(1)
memory usage: 15.1 MB
None


### Разделение данных
Преобразуем выборки в отдельные наборы данных

In [18]:
data_train, data_test = train_test_split(data_transformed,
                                         test_size=0.2)
data_train = pd.DataFrame(data_train)
data_test = pd.DataFrame(data_test)
print (data_train.head())

              0         1         2         3         4         5         6  \
8482  -1.236328 -0.164551 -2.841797 -0.892090 -0.083679  0.441650 -0.149292   
29512 -0.011101 -0.164551  0.312256  2.375000 -0.083679 -2.263672 -0.149292   
29210 -0.028320 -0.164551  0.312256  0.560059 -0.083679  0.441650 -0.149292   
12988 -0.976074 -0.164551  0.312256  1.467773 -0.083679  0.441650 -0.149292   
15046 -0.852539 -0.164551 -2.841797 -0.892090 -0.083679  0.441650 -0.149292   

              7         8         9  ...       124       125       126  \
8482   0.819336  0.269287 -0.630859  ... -0.142456 -0.240112  0.961914   
29512  1.803711  1.003906  0.144409  ... -0.142456 -0.240112 -1.887695   
29210  0.592285 -2.179688 -1.641602  ... -0.142456 -0.240112  0.961914   
12988  0.895020  1.003906  1.130859  ... -0.142456 -0.240112  0.147827   
15046  0.819336 -0.955078 -0.372559  ... -0.142456  4.164062  0.147827   

            127       128       129       130       131      132  Response  
848

### SVM
Выбираем направления преобразований исходных данных, чтобы различные классы можно было разделить гиперплоскостью по значениям параметров.
![](https://static.learme.ru/storage/uploads/editor/npnJ9VVXcfzhDGrl5e1IoPhSUTT5KmZL8seNRykU.png)

In [19]:
x = pd.DataFrame(data_train, columns=columns_transformed)
model_lin = LinearSVC(max_iter=10000)
model_lin.fit(x, data_train['Response'])

c:\users\nikolay\appdata\local\programs\python\python37\lib\site-packages\sklearn\svm\_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=10000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [20]:
model_sgd = SGDClassifier()
model_sgd.fit(x, data_train['Response'])

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge',
              max_iter=1000, n_iter_no_change=5, n_jobs=None, penalty='l2',
              power_t=0.5, random_state=None, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

### Предсказание данных и оценка модели

In [21]:
x_test = pd.DataFrame(data_test, columns=columns_transformed)
data_test["target_lin"] = model_lin.predict(x_test)
data_test["target_sgd"] = model_sgd.predict(x_test)

Кластеризация дает 0.192, kNN(100) - 0.3, лог. регрессия - 0.512/0.496

In [22]:
print ("SVM (линейный):",
       round(cohen_kappa_score(data_test["target_lin"],
                    data_test["Response"], weights='quadratic'), 3))
print ("SVM (градиент):",
       round(cohen_kappa_score(data_test["target_sgd"],
                    data_test["Response"], weights='quadratic'), 3))

SVM (линейный): 0.95
SVM (градиент): 0.914


### Матрица неточностей

In [23]:
print ("SVM (линейный)\n",
    confusion_matrix(data_test["target_lin"], data_test["Response"]))
print ("SVM (градиент)\n",
    confusion_matrix(data_test["target_sgd"], data_test["Response"]))

SVM (линейный)
 [[1244    0    0    0    0    0    0    0]
 [   0 1205   54   21   82   15    0    0]
 [   0    7    6    0    0    0    0    0]
 [   0   20   37  105    2   32    5    0]
 [   0   43   15    3  479  215  105    0]
 [   0   77   98  151  372 1515  614    0]
 [   0    0    0    3  148  403  907    0]
 [   0    0    0    0    0    0    0 3894]]
SVM (градиент)
 [[1237    0    0    0    0    0    0    0]
 [   7 1042   63   30  161   76   18    0]
 [   0   24   15   12    3    2    1    0]
 [   0   36   30   87    6   79   13    0]
 [   0   91   20   21  385  334  155    0]
 [   0  128   74  122  312 1169  593    0]
 [   0   31    8   11  216  520  851    0]
 [   0    0    0    0    0    0    0 3894]]
